<a href="https://colab.research.google.com/github/FatemehRanjbar/Image_process/blob/main/Persian_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract opencv-python
!sudo apt-get install tesseract-ocr-fas
!pip install pytesseract opencv-python pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,959 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
# 1. Install the required libraries
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!pip install pytesseract opencv-python-headless pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.3 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB

In [3]:

# 2. Download the Farsi trained data and place it in the correct directory
!wget https://github.com/tesseract-ocr/tessdata/raw/main/fas.traineddata
!mv fas.traineddata /usr/share/tesseract-ocr/4.00/tessdata/


--2024-11-21 15:09:09--  https://github.com/tesseract-ocr/tessdata/raw/main/fas.traineddata
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/fas.traineddata [following]
--2024-11-21 15:09:09--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/fas.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 561272 (548K) [application/octet-stream]
Saving to: ‘fas.traineddata’

fas.traineddata     100%[===================>] 548.12K  --.-KB/s    in 0.07s   

2024-11-21 15:09:10 (7.98 MB/s) - ‘fas.traineddata’ saved [561272/561272]



In [ ]:

import cv2
import pytesseract
from pytesseract import Output
from google.colab.patches import cv2_imshow
from google.colab import files
import itertools


uploaded = files.upload()


image_path = list(uploaded.keys())[0]


image = cv2.imread(image_path)


def rotate_image(image, angle):
    (h, w) = image.shape[:2]  # Get image dimensions
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)  # Calculate rotation matrix
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated


def ocr_image(image, lang='fas'):
    custom_config = r'--oem 3 --psm 6 -l ' + lang
    details = pytesseract.image_to_data(image, output_type=Output.DICT, config=custom_config)
    return details


def get_orientation(image):
    angles = range(0, 360, 10)  # Try rotating in 10-degree increments
    ocr_results = []

    # Loop through angles to find the angle with the maximum number of characters recognized
    for angle in angles:
        rotated = rotate_image(image, angle)
        details = ocr_image(rotated, lang='fas')
        num_chars = sum(len(word) for word in details['text'] if isinstance(word, str))
        ocr_results.append((angle, num_chars))

    # Find the angle with the highest character count
    best_angle = max(ocr_results, key=lambda x: x[1])[0]

    # Refine the angle by checking within a small range around the best angle found
    fine_angles = range(max(0, best_angle - 5), min(360, best_angle + 5))
    for angle in fine_angles:
        rotated = rotate_image(image, angle)
        details = ocr_image(rotated, lang='fas')
        num_chars = sum(len(word) for word in details['text'] if isinstance(word, str))
        if num_chars > max(ocr_results, key=lambda x: x[1])[1]:  # Update best angle if more characters are found
            best_angle = angle

    return best_angle


correct_angle = get_orientation(image)
print(f"Best angle: {correct_angle}")


rotated_image = rotate_image(image, correct_angle)


cv2.imwrite('rotated_image.jpg', rotated_image)


# cv2_imshow(rotated_image)


Saving example_image_2.jpg to example_image_2.jpg
[(0, 395), (10, 415), (20, 966), (30, 570), (40, 210), (50, 117), (60, 175), (70, 251), (80, 244), (90, 195), (100, 183), (110, 152), (120, 174), (130, 225), (140, 186), (150, 231), (160, 251), (170, 261), (180, 230), (190, 183), (200, 425), (210, 144), (220, 151), (230, 145), (240, 223), (250, 225), (260, 186), (270, 197), (280, 160), (290, 152), (300, 156), (310, 148), (320, 192), (330, 229), (340, 300), (350, 295)]
Best angle: 24


True

In [ ]:
#extract text from image (persian OCR)

In [4]:
!pip install pdf2image
!pip install pytesseract pillow
!pip install spellchecker
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 29.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for spellchecker: filename=spellchecker-0.4-py3-none-any.whl size=3966499 sha256=d3ec73e0a419cabe0c9777c14c0d410e9a4379030962479876af1c753910efe3
  Stored in directory: /root/.cache/pip/wheels/6c/90/c3/eac248d8755b2a7343487a2087b4b29ad98f388c3c8c69c286
  Created wheel for inexactsearch: filename=inexactsearch-1.0.2-py3-none-any.whl size=7119 sha256=d7cfa01b457cc415ab2de4d662d17a8f0d6cdc380adfe419e6534950d4e5ca76
  Stored in directory: /root/.cache/pip/wheels/63/19/2c/5e9f447f2533d457a1167c3e553f235e232b8a639e3f5fafab
  Created wheel for silpa_common: filename=silpa_common-0.3-py3-none-any.whl size=8469 sha256=d7a14b84f3fd4b4360faf655e48ded8baac8594ecb5b777dfe22ac561c0574fc
  Stored in directory: /root/.cache/pip/wheels/c0/72/43/0c7

In [19]:
import os
import re
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
from spellchecker import SpellChecker



# Preprocessing functions
def process_image(img_path):
    temp_filename = resize_image(img_path)
    img = remove_noise_and_smooth(temp_filename)
    return img

def resize_image(img_path):
    try:
        img = Image.open(img_path)
        length_x, width_y = img.size
        factor = max(1, int(1800 / length_x))  # 1800 for tesseract
        size = factor * length_x, factor * width_y
        im_resized = img.resize(size, Image.Resampling.LANCZOS)  # Use LANCZOS instead of ANTIALIAS

        import tempfile
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".TIFF")
        temp_filename = temp_file.name
        im_resized.save(temp_filename, dpi=(300, 300))  # Best for OCR

        return temp_filename
    except IOError:
        print("Error while reading the file.")


def remove_noise_and_smooth(img_path):
    try:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kernel = np.ones((1, 1), np.uint8)
        opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        img = cv2.bitwise_or(img, closing)
        return img
    except IOError:
        print("Error while reading the file.")



def correct_ocr_output(text):

    text = text.replace("ي", "ی").replace("ك", "ک")

    text = text.translate(str.maketrans("0123456789", "۰۱۲۳۴۵۶۷۸۹"))

    text = re.sub(r'\s+', ' ', text).strip()

    text = text.replace("،", ",").replace("؛", ";").replace("؟", "?")

    return text



# Main OCR processing script
langs = "fas"
input_dir = "/content"
output_dir = "/content"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def main():
    # Process PDF files
    for filename in os.listdir(input_dir):
        if filename.endswith('.pdf'):
            fullName = os.path.join(input_dir, filename)
            pages = convert_from_path(fullName, 500)
            image_counter = 1
            for page in pages:
                image_name = os.path.splitext(fullName)[0] + '_' + str(image_counter) + '.tiff'
                page.save(image_name, format='TIFF')
                image_counter += 1

    # Process image files
    for filename in os.listdir(input_dir):
        img_ext = ['.png', '.jpg', '.jpeg', '.tiff']
        if filename.endswith(tuple(img_ext)):
            fileAddress = os.path.join(input_dir, filename)
            img = process_image(fileAddress)
            config = '--psm 6'
            text = str(pytesseract.image_to_string(img, lang=langs, config=config))
            text = os.linesep.join([s for s in text.splitlines() if s.strip()])
            write_output(filename, text)


def write_output(filename, text):
    text = correct_ocr_output(text)
    outfile = os.path.join(output_dir, f'out_{os.path.splitext(filename)[0]}.txt')
    with open(outfile, 'w', encoding='utf-8') as text_file:
        text_file.write(text)


if __name__ == "__main__":
    main()


In [16]:
!pip install Levenshtein
!pip install jiwer

In [22]:
#calculate accuracy

import Levenshtein

def read_file(file_path):

    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


def calculate_ocr_accuracy(original_text, ocr_output):

    total_chars = len(original_text)
    edit_distance = Levenshtein.distance(original_text, ocr_output)
    char_accuracy = ((total_chars - edit_distance) / total_chars) * 100
    return char_accuracy


original_file = "original.txt"
ocr_output_file = "out_original.txt"


original_text = read_file(original_file)
ocr_output = read_file(ocr_output_file)

# original_text = preprocess_text(original_text)
# ocr_output = preprocess_text(ocr_output)


accuracy = calculate_ocr_accuracy(original_text, ocr_output)


print(f"OCR Accuracy: {accuracy:.2f}%")


OCR Accuracy: 98.34%
